# Initialization

Test notebook for the C-MAPPS benchmark. Get best parameters for each dataset. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import custom_scores
import CMAPSAuxFunctions
from tunableModel import TunableModel

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K

from scipy.optimize import differential_evolution

C:\Users\controlslab\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define the model

In [2]:
def RULmodel_SN(input_shape):
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(30, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', name='fc1'))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal', name='fc2'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Define objective function

Define the function that evaluates each set of data-related params and returns the RMSE as value.

In [5]:
def nn_optmize_fun(x, selected_features=['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32'], datasetNumber = '1', scaler = None, verbose=0, epochs=250, 
                  saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)
    #windows_size = x[0]
    windows_size = maxWindowSize[datasetNumber]
    window_stride = x[0]
    constantRUL = x[1]
    
    if iterations == 0:
        print("Creating model")
    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    
    #Define the model
    nFeatures = len(selected_features)
    shapeSN = nFeatures*windows_size
    modelRULSN = RULmodel_SN(shapeSN)
    modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    #load the data using the selected parameters
    tModel = TunableModel("RUL_SN_optmizable", modelRULSN, selected_features, '../CMAPSSData', 'keras',
                          scaler = min_max_scaler, window_stride = window_stride, 
                          window_size = windows_size, constRul = constantRUL, datasetNumber = datasetNumber, 
                         epochs=epochs)
    
    tModel.loadData(rectify_labels = False)
    
    if iterations == 0:
        print("Training model")
    #Train the model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    tModel.trainModel(learningRateScheduler=lrate, verbose=verbose)
    time = tModel.trainTime
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluateModel(["rhs"], round=2, scalingFactor=20)
    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    
    if saveToFile is not None:
        #print(msgStr)
        saveToFile.write(msgStr)
    else:
        print(msgStr)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

# Optimize the parameters for the NN using DE

In [8]:
#Optimize the parameters for the NN using DE

#maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
maxWindowSize = {'1':30, '2':20} #Do it only for datasets 1 and 2
totalTime = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

#datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("intermediateResults4_notRectified3"+datasetNumber+".txt", "w")

    windowSizeBounds = [1,maxWindowSize[datasetNumber]]
    windowStrideBounds = [1,10]
    constantRULBounds = [90,120]

    #bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]
    bounds = [windowStrideBounds, constantRULBounds] #Optimize only 2 variabes


    startTime = time.clock()
    tempResults = differential_evolution(nn_optmize_fun, bounds, 
                                     args=(selected_features, datasetNumber, min_max_scaler, 0, 20, file, 1),
                                    strategy='best1bin', maxiter=10, popsize=20, disp=True)
    results[datasetNumber] = tempResults
    endTime = time.clock()

    file.close()
    totalTime[datasetNumber] = endTime - startTime

Tunning for dataset 1
100/100 [==============================] - 0s 249us/step
differential_evolution step 1: f(x)= 16.4852
100/100 [==============================] - 0s 249us/step
differential_evolution step 2: f(x)= 16.11
100/100 [==============================] - 0s 249us/step
differential_evolution step 3: f(x)= 16.11
100/100 [==============================] - 0s 250us/step
differential_evolution step 4: f(x)= 16.0693
100/100 [==============================] - 0s 249us/step
differential_evolution step 5: f(x)= 15.8821
100/100 [==============================] - 0s 259us/step
differential_evolution step 6: f(x)= 15.8821
100/100 [==============================] - 0s 249us/step


100/100 [==============================] - 0s 249us/step
differential_evolution step 7: f(x)= 15.8821
100/100 [==============================] - 0s 249us/step
differential_evolution step 8: f(x)= 15.7933
100/100 [==============================] - 0s 270us/step
differential_evolution step 9: f(x)= 15.7933
100/100 [==============================] - 0s 259us/step
differential_evolution step 10: f(x)= 15.7933
100/100 [==============================] - 0s 249us/step
Tunning for dataset 2
259/259 [==============================] - 0s 116us/step


259/259 [==============================] - 0s 112us/step
differential_evolution step 1: f(x)= 35.0289
259/259 [==============================] - 0s 116us/step
differential_evolution step 2: f(x)= 35.0289
259/259 [==============================] - 0s 119us/step
differential_evolution step 3: f(x)= 35.0202
259/259 [==============================] - 0s 119us/step
differential_evolution step 4: f(x)= 34.7929
259/259 [==============================] - 0s 123us/step
differential_evolution step 5: f(x)= 34.7929
259/259 [==============================] - 0s 116us/step
differential_evolution step 6: f(x)= 34.7929
259/259 [==============================] - 0s 112us/step
differential_evolution step 7: f(x)= 34.7482
259/259 [==============================] - 0s 116us/step


259/259 [==============================] - 0s 123us/step
differential_evolution step 8: f(x)= 34.7482
259/259 [==============================] - 0s 112us/step
differential_evolution step 9: f(x)= 34.7482
259/259 [==============================] - 0s 127us/step
differential_evolution step 10: f(x)= 34.7482
259/259 [==============================] - 0s 112us/step


In [9]:
print("Total time {}".format(totalTime))
print(results)

Total time {'1': 1818.575037156521, '4': 0, '3': 0, '2': 4566.766281821978}
{'1':      fun: 15.79325127669005
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 491
     nit: 10
 success: False
       x: array([  1.5814512 , 119.08023773]), '4': 0, '3': 0, '2':      fun: 34.74818561094152
 message: 'Optimization terminated successfully.'
    nfev: 527
     nit: 10
 success: True
       x: array([  1.9988995 , 118.79595482])}
